In [1]:
%load_ext autoreload
%autoreload 2

# Export cells
!python notebook2script.py layers.ipynb

Converted models/layers.py to torchtrainer/models/layers.py


In [1]:
# Cross-hair convolution filters
#export models/layers.py
from collections.abc import Iterable
import torch
import torch.nn as nn

def ntuple(x, n):
    '''Verify if x is iterable. If not, create tuple containing x repeated n times'''
    
    if isinstance(x, Iterable):
        return x
    return tuple([x]*n)

class Conv2dCH(nn.Module):
    """Create 2D cross-hair convolution filter. Parameters are the same as torch.nn.Conv2d, with the exception
    that padding must be larger than or equal to (kernel_size-1)//2 (otherwise the filter would need negative padding
    to properly work) and dilation is not supported. Also, if padding is not provided it will be equal to (kernel_size-1)//2.
    That is, by default the result of the convolution has the same shape as the input tensor.

    Parameters
    ----------
    in_channels : int
        Number of input channels. 
    out_channels : int
        Number of output channels. 
    kernel_size : int or tuple
        Size of the kernel. Even sizes are not supported. 
    stride : int or tuple
        Stride of the convolution.
    padding : int or tuple
        Padding of the input. Must be larger than or equal to (kernel_size-1)//2.
    groups : int
        Controls the connections between inputs and outputs. 
    bias : bool
        If True, adds a learnable bias to the output.
    padding_mode : string
        Padding mode to use.
    """    

    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=None, groups=1, bias=True, 
                 padding_mode='zeros'):
        super(Conv2dCH, self).__init__()
        
        kernel_size = ntuple(kernel_size, 2)
        stride = ntuple(stride, 2)
        padding = ntuple(padding, 2)
        
        padding = list(padding)
        if padding[0] is None:
            padding[0] = (kernel_size[0]-1)//2
        if padding[1] is None:
            padding[1] = (kernel_size[1]-1)//2
        
        if padding[0]<(kernel_size[0]-1)//2:
            raise ValueError("Padding must be padding[0]>=(kernel_size[0]-1)//2")
        if padding[1]<(kernel_size[1]-1)//2:
            raise ValueError("Padding must be padding[1]>=(kernel_size[1]-1)//2")
            
        pad_conv1d_v_h = padding[1]-(kernel_size[1]-1)//2
        pad_conv1d_h_v = padding[0]-(kernel_size[0]-1)//2
        self.conv1d_v = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, 
                                 kernel_size=(kernel_size[0], 1), stride=stride, padding=(padding[0], pad_conv1d_v_h), 
                                 groups=groups, bias=bias, padding_mode=padding_mode) 
        self.conv1d_h = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, 
                                 kernel_size=(1, kernel_size[1]), stride=stride, padding=(pad_conv1d_h_v, padding[1]), 
                                 groups=groups, bias=bias, padding_mode=padding_mode)    
        
        self.reset_parameters()

    def forward(self, x):
        return self.conv1d_v(x) + self.conv1d_h(x)

    def reset_parameters(self):
        
        self.conv1d_v.reset_parameters()
        self.conv1d_h.reset_parameters()

        '''for module in self.modules():
            if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
                nn.init.kaiming_normal_(module.weight)
                if module.bias is not None:
                    module.bias.data.zero_()'''
        
class Conv3dCH(nn.Module):
    """Create 3D cross-hair convolution filter. Parameters are the same as torch.nn.Conv3d, with the exception
    that padding must be larger than or equal to (kernel_size-1)//2 (otherwise the filter would need negative padding
    to properly work) and dilation is not supported. Also, if padding is not provided it will be equal to (kernel_size-1)//2.
    That is, by default the result of the convolution has the same shape as the input tensor.

    Parameters
    ----------
    in_channels : int
        Number of input channels. 
    out_channels : int
        Number of output channels. 
    kernel_size : int or tuple
        Size of the kernel. Even sizes are not supported. 
    stride : int or tuple
        Stride of the convolution.
    padding : int or tuple
        Padding of the input. Must be larger than or equal to (kernel_size-1)//2.
    groups : int
        Controls the connections between inputs and outputs. 
    bias : bool
        If True, adds a learnable bias to the output.
    padding_mode : string
        Padding mode to use.
    """    

    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=None, groups=1, bias=True, 
                 padding_mode='zeros'):
        super(Conv3dCH, self).__init__()
        
        kernel_size = ntuple(kernel_size, 3)
        stride = ntuple(stride, 3)
        padding = ntuple(padding, 3)
        
        padding = list(padding)
        if padding[0] is None:
            padding[0] = (kernel_size[0]-1)//2
        if padding[1] is None:
            padding[1] = (kernel_size[1]-1)//2
        if padding[2] is None:
            padding[2] = (kernel_size[2]-1)//2
        
        if padding[0]<(kernel_size[0]-1)//2:
            raise ValueError("Padding must be padding[0]>=(kernel_size[0]-1)//2")
        if padding[1]<(kernel_size[1]-1)//2:
            raise ValueError("Padding must be padding[1]>=(kernel_size[1]-1)//2")
        if padding[2]<(kernel_size[2]-1)//2:
            raise ValueError("Padding must be padding[2]>=(kernel_size[2]-1)//2")
            
        pad_conv2d_p_v = pad_conv2d_h_v = padding[1]-(kernel_size[1]-1)//2
        pad_conv2d_p_h = pad_conv2d_v_h = padding[2]-(kernel_size[2]-1)//2
        pad_conv2d_v_p = pad_conv2d_h_p = padding[0]-(kernel_size[0]-1)//2

        self.conv2d_p = nn.Conv3d(in_channels=in_channels, out_channels=out_channels, 
                                 kernel_size=(kernel_size[0], 1, 1), stride=stride, 
                                 padding=(padding[0], pad_conv2d_p_v, pad_conv2d_p_h), 
                                 groups=groups, bias=bias, padding_mode=padding_mode) 
        self.conv2d_v = nn.Conv3d(in_channels=in_channels, out_channels=out_channels, 
                                 kernel_size=(1, kernel_size[1], 1), stride=stride, 
                                 padding=(pad_conv2d_v_p, padding[1], pad_conv2d_v_h), 
                                 groups=groups, bias=bias, padding_mode=padding_mode) 
        self.conv2d_h = nn.Conv3d(in_channels=in_channels, out_channels=out_channels, 
                                 kernel_size=(1, 1, kernel_size[2]), stride=stride, 
                                 padding=(pad_conv2d_h_p, pad_conv2d_h_v, padding[2]), 
                                 groups=groups, bias=bias, padding_mode=padding_mode) 
        
        self.reset_parameters()

    def forward(self, x):
        return self.conv2d_p(x) + self.conv2d_v(x) + self.conv2d_h(x)

    def reset_parameters(self):
        
        self.conv2d_p.reset_parameters()
        self.conv2d_v.reset_parameters()
        self.conv2d_h.reset_parameters()

        '''for module in self.modules():
            if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
                nn.init.kaiming_normal_(module.weight)
                if module.bias is not None:
                    module.bias.data.zero_()'''   

In [4]:
#Conv2dCH(1, 2, (3,3), stride=1, bias=False)(img[None, None])   
inp = torch.rand(1, 1, 5, 5, 5)
res = Conv3dCH(1, 1, (3, 3, 3), stride=1, bias=False)(inp) 

In [14]:
import time

class TestModule(nn.Module):

    def __init__(self, layer, nlayers, channels, int_channels=16, kernel_size=3):
        super(TestModule, self).__init__()
        
        kernel_size = ntuple(3, 3)
        
        layers = [layer(channels, int_channels, kernel_size)]
        for i in range(nlayers):
            layers.append(layer(int_channels, int_channels, kernel_size))
        layers.append(layer(int_channels, 1, kernel_size))
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x):
        
        for layer in self.children():
            x = layer(x)
        return x

def time_model(model, num_iterations=10, bs=8, channels=32, shape=None):
    
    if shape is None:
        shape = (50, 50)
        
    model.cuda()

    tt = 0
    for i in range(num_iterations):
        inp = torch.rand(bs, channels, *shape).cuda()
        ti = time.time()
        res = model(inp)
        tf = time.time()
        tt += tf-ti
    tm = tt/num_iterations
    
    model.cpu()

    return tm


conv3d = nn.Conv3d(3, 32, (7, 7, 7))
conv3dch = Conv3dCH(3, 32, (7, 7, 7))
tm = TestModule(nn.Conv3d, nlayers=18, channels=1, int_channels=32)

res = time_model(tm, num_iterations=10, bs=2, channels=1, shape=(70, 70, 70))

In [15]:
res

0.0020944833755493163

TestModule()